<a href="https://colab.research.google.com/github/Amirtha2005/Glaucoma-Detection-Using-CNN-Feature-Extraction/blob/main/assignment2_glaucoma_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving glaucoma (1).zip to glaucoma (1) (1).zip


In [ ]:
import zipfile
import os
with zipfile.ZipFile('glaucoma.zip', 'r') as zip_ref:
    zip_ref.extractall('glaucoma_data')


In [ ]:
import cv2
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from scipy.stats import entropy
import os

def extract_features(img_path):
    img = cv2.imread(img_path)
    if img is None:
        return None
    img = cv2.resize(img, (224, 224))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    energy = np.sum(gray ** 2) / gray.size
    ent = entropy(np.histogram(gray, bins=256)[0] + 1)
    brightness = np.mean(gray)
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()

    glcm = graycomatrix(gray, [1], [0], 256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]

    return [energy, ent, brightness, sharpness, contrast, correlation]


In [ ]:
def process_dataset(data_dir):
    data = []
    labels = []

    for label in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, label)
        if not os.path.isdir(folder_path):
            continue
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            features = extract_features(img_path)
            if features:
                data.append(features)
                labels.append(label)

    return np.array(data), np.array(labels)

data_dir = 'glaucoma_data/train'
X, y = process_dataset(data_dir)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical


le = LabelEncoder()
y_encoded = to_categorical(le.fit_transform(y))


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_encoded.shape[1], activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5427 - loss: 3.0704 - val_accuracy: 0.5600 - val_loss: 0.7246
Epoch 2/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5262 - loss: 0.9365 - val_accuracy: 0.4737 - val_loss: 1.2804
Epoch 3/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5532 - loss: 0.7854 - val_accuracy: 0.5450 - val_loss: 0.7111
Epoch 4/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5322 - loss: 0.7613 - val_accuracy: 0.5375 - val_loss: 0.7727
Epoch 5/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5447 - loss: 0.7348 - val_accuracy: 0.5462 - val_loss: 0.7050
Epoch 6/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5249 - loss: 0.8200 - val_accuracy: 0.5813 - val_loss: 0.6862
Epoch 7/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5233 - loss: 0.8667 - val_accuracy: 0.5575 - val_loss: 0.6871
Epoch 8/30
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5435 - loss: 0.8089 - val_accuracy: 0.

In [ ]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc*100:.2f}%")


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5293 - loss: 0.7695
Test Accuracy: 50.20%


In [ ]:
df = pd.DataFrame(X, columns=['Energy', 'Entropy', 'Brightness', 'Sharpness', 'Contrast', 'Correlation'])
df['Label'] = y
df.to_excel('glaucoma_features.xlsx', index=False)


In [ ]:
files.download('glaucoma_features.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

df = pd.read_excel("glaucoma_features.xlsx")


X = df.drop("Label", axis=1)
y = df["Label"]


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
label_map = {'NRG': 0, 'RG': 1}
y = df['Label'].map(label_map)

In [ ]:
X = df.drop("Label", axis=1)
X = X.astype(float)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [ ]:
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5476 - loss: 0.6909 - val_accuracy: 0.6020 - val_loss: 0.6779
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5744 - loss: 0.6802 - val_accuracy: 0.6060 - val_loss: 0.6657
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5866 - loss: 0.6794 - val_accuracy: 0.6180 - val_loss: 0.6607
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.5986 - loss: 0.6708 - val_accuracy: 0.6090 - val_loss: 0.6624
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5792 - loss: 0.6734 - val_accuracy: 0.6340 - val_loss: 0.6544
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6013 - loss: 0.6654 - val_accuracy: 0.6180 - val_loss: 0.6562
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6178 - loss: 0.6616 - val_accuracy: 0.6330 - val_loss: 0.6544
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5962 - loss: 0.6695 - val_accuracy: 0.

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6488 - loss: 0.6300
Test Accuracy: 0.65
